In [1]:
# Import Libraries
import mlflow
import pandas as pd
import xgboost
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error


In [2]:
# defining a function to quickly read and prepare data
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    df['hour'] = df.lpep_pickup_datetime.dt.hour
    df['dayofweek'] = df.lpep_pickup_datetime.dt.day_of_week

    categorical = ['PULocationID', 'DOLocationID', 'hour', 'dayofweek', 'VendorID']
    df[categorical] = df[categorical].astype(str)
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    
    return df

In [3]:
# Reading and preparing the dataset
df_train = read_dataframe('/home/ubuntu/data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('/home/ubuntu/data/green_tripdata_2021-02.parquet')

In [4]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment-xgboost")

<Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='nyc-taxi-experiment-xgboost', tags={}>

In [5]:
# defing feature types
categorical = ['PU_DO', 'hour', 'dayofweek', 'VendorID'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

# applying one hot encoding
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [6]:
# defining the target
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [7]:
# defining xboost level data

training_data = xgboost.DMatrix(X_train, label=y_train)
valid = xgboost.DMatrix(X_val, label=y_val)

In [8]:
def objective(params):
    
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        model = xgboost.train(
            params = params,
            dtrain = training_data,
            num_boost_round = 100,
            evals = [(valid, "validation")],
            early_stopping_rounds = 20
        )

        y_pred = model.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -6, 1),
    'reg_lambda': hp.loguniform('reg_lambda', -5, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective' : 'reg:squarederror',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)



[0]	validation-rmse:19.57378                          
[1]	validation-rmse:18.10962                          
[2]	validation-rmse:16.77918                          
[3]	validation-rmse:15.58026                          
[4]	validation-rmse:14.49962                          
[5]	validation-rmse:13.51971                          
[6]	validation-rmse:12.63830                          
[7]	validation-rmse:11.84852                          
[8]	validation-rmse:11.13480                          
[9]	validation-rmse:10.50404                          
[10]	validation-rmse:9.93552                          
[11]	validation-rmse:9.43576                          
[12]	validation-rmse:8.99226                          
[13]	validation-rmse:8.60027                          
[14]	validation-rmse:8.25943                          
[15]	validation-rmse:7.95904                          
[16]	validation-rmse:7.70254                          
[17]	validation-rmse:7.47771                          
[18]	valid

In [11]:
# Train the best model
best_result['max_depth'] = int(best_result['max_depth'])
best_result['objective'] = 'reg:squarederror'
best_result['seed'] = 42

#mlflow.xgboost.autolog()
with mlflow.start_run():
    mlflow.set_tag("model", "xgboost")
    mlflow.log_params(best_result)
    model = xgboost.train(
            params = best_result,
            dtrain = training_data,
            num_boost_round = 200,
            evals = [(valid, "validation")],
            early_stopping_rounds = 50
        )
    y_pred = model.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)


2022/05/25 14:10:29 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'f093999e916346c0a2a15afed2a5be4d', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:19.57378
[1]	validation-rmse:18.10962
[2]	validation-rmse:16.77918
[3]	validation-rmse:15.58026
[4]	validation-rmse:14.49962
[5]	validation-rmse:13.51971
[6]	validation-rmse:12.63830
[7]	validation-rmse:11.84852
[8]	validation-rmse:11.13480
[9]	validation-rmse:10.50404
[10]	validation-rmse:9.93552
[11]	validation-rmse:9.43576
[12]	validation-rmse:8.99226
[13]	validation-rmse:8.60027
[14]	validation-rmse:8.25943
[15]	validation-rmse:7.95904
[16]	validation-rmse:7.70254
[17]	validation-rmse:7.47771
[18]	validation-rmse:7.28234
[19]	validation-rmse:7.11537
[20]	validation-rmse:6.97207
[21]	validation-rmse:6.85085
[22]	validation-rmse:6.74673
[23]	validation-rmse:6.65659
[24]	validation-rmse:6.58096
[25]	validation-rmse:6.51724
[26]	validation-rmse:6.46401
[27]	validation-rmse:6.41703
[28]	validation-rmse:6.37728
[29]	validation-rmse:6.34179
[30]	validation-rmse:6.31142
[31]	validation-rmse:6.28592
[32]	validation-rmse:6.26396
[33]	validation-rmse:6.24707
[34]	validatio

2022/05/25 14:11:45 ERROR mlflow.xgboost: Failed to log feature importance plot. XGBoost autologging will ignore the failure and continue. Exception: 
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/mlflow/xgboost/__init__.py", line 637, in train
    log_feature_importance_plot(features, importance, imp_type)
  File "/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/mlflow/xgboost/__init__.py", line 549, in log_feature_importance_plot
    fig.tight_layout()
  File "/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/matplotlib/figure.py", line 3222, in tight_layout
    renderer = get_renderer(self)
  File "/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/matplotlib/tight_layout.py", line 211, in get_renderer
    return canvas.get_renderer()
  File "/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/matplotlib/backends/backend_agg.

ValueError: Image size of 460x79048 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 460.8x79048.8 with 1 Axes>